In [19]:
import json, time
import pyspark
import os
import re

In [20]:
from pyspark.sql.types import DataType, BooleanType, NullType, IntegerType, StringType, MapType

In [21]:
from pyspark.sql.functions import udf, explode, to_date

# Getting file names

In [22]:
DIR = 'archive'
dir_list = []
dir_list += [os.path.join(DIR,file) for file in os.listdir(DIR) if os.path.isdir(os.path.join(DIR, file))]
dir_list.sort()

In [5]:
# Get latest quarter
quarter_folder = dir_list[-2]

In [6]:
json_files = []
json_files += [os.path.join(quarter_folder, file) for file in os.listdir(quarter_folder)]

In [7]:
json_file = json_files[0]
json_file

'archive/2022.QTR2/0001493152-22-013349.json'

# Initialize PySpark session

In [8]:
spark = pyspark.sql.SparkSession.builder \
    .appName("Decode_json_files") \
    .config("spark.jars", "postgresql-42.5.1.jar") \
    .getOrCreate()

23/01/23 21:08:03 WARN Utils: Your hostname, Ignacios-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.230 instead (on interface en0)
23/01/23 21:08:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/23 21:08:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/23 21:08:03 WARN DependencyUtils: Local jar /Users/ignaciomoyaredondo/Documents/data/pyspark_financials/postgresql-42.5.1.jar does not exist, skipping.
23/01/23 21:08:04 INFO SparkContext: Running Spark version 3.3.1
23/01/23 21:08:04 INFO ResourceUtils: ==============================================================
23/01/23 21:08:04 INFO ResourceUtils: No custom resources configured for spark.driver.
23/01/23 21:08:04 INFO ResourceUtils: ==============================================================
23/01/23 21:08:04 INFO SparkContext: Submitted application: Decode_json_files
23/01/23 21:08:04

# Data Exploration

In [9]:
%time
df = spark.read.option("multiline", "true").json(json_file)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
23/01/23 21:08:05 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
23/01/23 21:08:05 INFO SharedState: Warehouse path is 'file:/Users/ignaciomoyaredondo/Documents/data/pyspark_financials/spark-warehouse'.
23/01/23 21:08:06 INFO InMemoryFileIndex: It took 24 ms to list leaf files for 1 paths.
23/01/23 21:08:06 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 356.0 KiB, free 366.0 MiB)
23/01/23 21:08:06 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.2 KiB, free 365.9 MiB)
23/01/23 21:08:06 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 192.168.1.230:49853 (size: 34.2 KiB, free: 366.3 MiB)
23/01/23 21:08:06 INFO SparkContext: Created broadcast 0 from json at NativeMethodAccessorImpl.java:0
23/01/23 21:08:06 INFO FileInputFormat: Total input files to process : 1
23/01/23 21:08:06 INFO Fil

In [10]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- bs: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- concept: string (nullable = true)
 |    |    |    |-- label: string (nullable = true)
 |    |    |    |-- unit: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)
 |    |-- cf: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- concept: string (nullable = true)
 |    |    |    |-- label: string (nullable = true)
 |    |    |    |-- unit: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)
 |    |-- ic: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- concept: string (nullable = true)
 |    |    |    |-- label: string (nullable = true)
 |    |    |    |-- unit: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)
 |-- endDate: string (nullable = true)
 |-- quarter: string

In [18]:
df.show()

23/01/23 21:13:28 INFO FileSourceStrategy: Pushed Filters: 
23/01/23 21:13:28 INFO FileSourceStrategy: Post-Scan Filters: 
23/01/23 21:13:28 INFO FileSourceStrategy: Output Data Schema: struct<data: struct<bs: array<struct<concept:string,label:string,unit:string,value:string>>, cf: array<struct<concept:string,label:string,unit:string,value:string>>, ic: array<struct<concept:string,label:string,unit:string,value:string>> ... 1 more fields>, endDate: string, quarter: string, startDate: string, symbol: string ... 1 more field>
23/01/23 21:13:28 INFO CodeGenerator: Code generated in 249.525346 ms
23/01/23 21:13:28 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 350.3 KiB, free 366.0 MiB)
23/01/23 21:13:28 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 365.9 MiB)
23/01/23 21:13:28 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 192.168.1.230:49853 (size: 34.1 KiB, free: 366.3 MiB)
23/01/23 21:13

### Balance sheet spark dataframe

In [11]:
df_bs = df.withColumn("bs", explode('data.bs')).select('bs')

In [12]:
df_bs.printSchema()

root
 |-- bs: struct (nullable = true)
 |    |-- concept: string (nullable = true)
 |    |-- label: string (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- value: string (nullable = true)



In [13]:
df_bs = df_bs.select('bs.concept', 'bs.label', 'bs.unit', 'bs.value').distinct()

In [14]:
df_bs.show()

23/01/17 19:34:10 INFO FileSourceStrategy: Pushed Filters: 
23/01/17 19:34:10 INFO FileSourceStrategy: Post-Scan Filters: (size(data#0.bs, true) > 0),isnotnull(data#0.bs)
23/01/17 19:34:10 INFO FileSourceStrategy: Output Data Schema: struct<data: struct<bs: array<struct<concept:string,label:string,unit:string,value:string>>, cf: array<struct<concept:string,label:string,unit:string,value:string>>, ic: array<struct<concept:string,label:string,unit:string,value:string>> ... 1 more fields>>
23/01/17 19:34:10 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
23/01/17 19:34:10 INFO CodeGenerator: Code generated in 210.625113 ms
23/01/17 19:34:10 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 350.3 KiB, free 366.0 MiB)
23/01/17 19:34:10 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 365.9 MiB)

### Cash flow spark dataframe

In [15]:
df_cf = df.withColumn("bs", explode('data.cf')).select('bs')

In [16]:
df_cf.printSchema()

root
 |-- bs: struct (nullable = true)
 |    |-- concept: string (nullable = true)
 |    |-- label: string (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- value: string (nullable = true)



In [17]:
df_cf = df_cf.select('bs.concept', 'bs.label', 'bs.unit', 'bs.value').distinct()

In [18]:
df_cf.show()

23/01/17 19:34:11 INFO FileSourceStrategy: Pushed Filters: 
23/01/17 19:34:11 INFO FileSourceStrategy: Post-Scan Filters: (size(data#0.cf, true) > 0),isnotnull(data#0.cf)
23/01/17 19:34:11 INFO FileSourceStrategy: Output Data Schema: struct<data: struct<bs: array<struct<concept:string,label:string,unit:string,value:string>>, cf: array<struct<concept:string,label:string,unit:string,value:string>>, ic: array<struct<concept:string,label:string,unit:string,value:string>> ... 1 more fields>>
23/01/17 19:34:11 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
23/01/17 19:34:11 INFO CodeGenerator: Code generated in 33.387662 ms
23/01/17 19:34:11 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 350.3 KiB, free 365.5 MiB)
23/01/17 19:34:11 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 365.4 MiB)


### Income statement spark dataframe

In [19]:
df_ic = df.withColumn("ic", explode('data.ic')).select('ic')

In [20]:
df_ic.printSchema()

root
 |-- ic: struct (nullable = true)
 |    |-- concept: string (nullable = true)
 |    |-- label: string (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- value: string (nullable = true)



In [21]:
df_ic = df_ic.select('ic.concept', 'ic.label', 'ic.unit', 'ic.value').distinct()

In [22]:
df_ic.show()

23/01/17 19:34:11 INFO FileSourceStrategy: Pushed Filters: 
23/01/17 19:34:11 INFO FileSourceStrategy: Post-Scan Filters: (size(data#0.ic, true) > 0),isnotnull(data#0.ic)
23/01/17 19:34:11 INFO FileSourceStrategy: Output Data Schema: struct<data: struct<bs: array<struct<concept:string,label:string,unit:string,value:string>>, cf: array<struct<concept:string,label:string,unit:string,value:string>>, ic: array<struct<concept:string,label:string,unit:string,value:string>> ... 1 more fields>>
23/01/17 19:34:11 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
23/01/17 19:34:11 INFO CodeGenerator: Code generated in 33.83924 ms
23/01/17 19:34:11 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 350.3 KiB, free 365.0 MiB)
23/01/17 19:34:12 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 364.9 MiB)
2

### General information about the company

In [23]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- bs: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- concept: string (nullable = true)
 |    |    |    |-- label: string (nullable = true)
 |    |    |    |-- unit: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)
 |    |-- cf: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- concept: string (nullable = true)
 |    |    |    |-- label: string (nullable = true)
 |    |    |    |-- unit: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)
 |    |-- ic: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- concept: string (nullable = true)
 |    |    |    |-- label: string (nullable = true)
 |    |    |    |-- unit: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)
 |-- endDate: string (nullable = true)
 |-- quarter: string

In [24]:
df = df.withColumn('startDate', to_date(df.startDate, 'yyyy-MM-dd'))
df = df.withColumn('endDate',to_date(df.endDate, 'yyyy-MM-dd'))
df = df.withColumn('year', df.year.cast('int'))

In [25]:
year = df.select('year').collect()[0]#show()

23/01/17 19:34:12 INFO FileSourceStrategy: Pushed Filters: 
23/01/17 19:34:12 INFO FileSourceStrategy: Post-Scan Filters: 
23/01/17 19:34:12 INFO FileSourceStrategy: Output Data Schema: struct<year: string>
23/01/17 19:34:12 INFO CodeGenerator: Code generated in 6.940201 ms
23/01/17 19:34:12 INFO MemoryStore: Block broadcast_11 stored as values in memory (estimated size 350.3 KiB, free 364.5 MiB)
23/01/17 19:34:12 INFO MemoryStore: Block broadcast_11_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 364.4 MiB)
23/01/17 19:34:12 INFO BlockManagerInfo: Added broadcast_11_piece0 in memory on 192.168.1.230:54172 (size: 34.1 KiB, free: 366.1 MiB)
23/01/17 19:34:12 INFO SparkContext: Created broadcast 11 from collect at /var/folders/zf/p75y8jj52c72dh5cv6ys92pm0000gn/T/ipykernel_86379/2964144282.py:1
23/01/17 19:34:12 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
23/01/17 19:34:12 INFO SparkCo

In [26]:
df.show()

23/01/17 19:34:12 INFO FileSourceStrategy: Pushed Filters: 
23/01/17 19:34:12 INFO FileSourceStrategy: Post-Scan Filters: 
23/01/17 19:34:12 INFO FileSourceStrategy: Output Data Schema: struct<data: struct<bs: array<struct<concept:string,label:string,unit:string,value:string>>, cf: array<struct<concept:string,label:string,unit:string,value:string>>, ic: array<struct<concept:string,label:string,unit:string,value:string>> ... 1 more fields>, endDate: string, quarter: string, startDate: string, symbol: string ... 1 more field>
23/01/17 19:34:12 INFO CodeGenerator: Code generated in 28.154311 ms
23/01/17 19:34:12 INFO MemoryStore: Block broadcast_13 stored as values in memory (estimated size 350.3 KiB, free 364.1 MiB)
23/01/17 19:34:12 INFO MemoryStore: Block broadcast_13_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 364.0 MiB)
23/01/17 19:34:12 INFO BlockManagerInfo: Added broadcast_13_piece0 in memory on 192.168.1.230:54172 (size: 34.1 KiB, free: 366.0 MiB)
23/01/17 19:

# Track one company over time

In [27]:
def get_json_in_quarter(dir_quarter):
    json_files = []
    json_files += [os.path.join(quarter_folder, file) for file in os.listdir(quarter_folder)]
    return json_files

In [28]:
def get_ticker_in_json_pyspark(json_file):
    try:
        df = spark.read.option("multiline", "true").json(json_file)
        return df.select("symbol").collect()[0][0]
    except NameError:
        print('Please initialize Spark session in variable called "spark"')

Let's get a random ticker so that one can see how 

In [29]:
def get_ticker_in_json(json_file):
    try:
        with open(json_file, 'r') as jf:
            df = json.load(jf)
        return df['symbol']
    except IsADirectoryError:
        pass    

### Create database

In [30]:
import psycopg2
import configparser

In [31]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [32]:
DB_NAME = config['DB']['name']
DB_USER = config['DB']['user']
DB_PASS = config['DB']['pass']
DB_HOST = config['DB']['host']
DB_PORT = config['DB']['port']

In [33]:
try:
    conn = psycopg2.connect(database = DB_NAME, user = DB_USER, 
                            password = DB_PASS, host = DB_HOST,
                            port = DB_PORT)
    print("Connection to database successful")
except:
    print("Connection to database failed")

Connection to database successful


In [34]:
cur = conn.cursor()

In [35]:
### Get lists to be implemented in the database

In [36]:
def get_year_qtr_of_dir(filedir):
    year = int(re.search('archive/(.*).QTR', filedir).group(1))
    qtr = int(re.search('QTR(.*)', filedir).group(1))
    return year, qtr

In [39]:
tickers = []
for json_file in json_files[:100]:
    tickers.append(get_ticker_in_json(json_file))
tickers_id = list(enumerate(tickers))

In [37]:
filedir = dir_list[0]
year, qtr = get_year_qtr_of_dir(filedir)

In [38]:
json_files = get_json_in_quarter(filedir)

In [40]:
type(tickers_id[0][1])

str

In [41]:
cur.execute("""
            DROP TABLE IF EXISTS summary_archive;
            CREATE TABLE IF NOT EXISTS summary_archive (
            year INT NOT NULL,
            quarter INT NOT NULL, 
            symbol VARCHAR ( 50 ) NOT NULL,
            id INT NOT NULL
            )
            """)

In [42]:
cur.execute("""
            INSERT INTO summary_archive (year, quarter, symbol, id)
            VALUES (%s, %s, %s, %s);
            """, 
            (year, qtr, tickers_id[0][1], tickers_id[0][0]))

In [43]:
cur.execute("SELECT * FROM summary_archive")
a = cur.fetchall()
a

[(2009, 2, 'OGEN', 0)]

In [44]:
conn.commit()

#### Automate the summary archive table creation for a given quarter

In [45]:
cur.execute("""
            DROP TABLE IF EXISTS summary_archive;
            CREATE TABLE IF NOT EXISTS summary_archive (
            year INT NOT NULL,
            quarter INT NOT NULL, 
            symbol VARCHAR ( 50 ),
            id INT
            )
            """)

Note that the drop table only has to be executed at the beginning of the pipeline

In [46]:
for filedir in dir_list[:5]:
    json_files = get_json_in_quarter(filedir)
    tickers = []
    for json_file in json_files:
        tickers.append(get_ticker_in_json(json_file))
    tickers_id = list(enumerate(tickers))
    for ticker in tickers_id:
        cur.execute("""
                INSERT INTO summary_archive (year, quarter, symbol, id)
                VALUES (%s, %s, %s, %s);
                """, 
                (year, qtr, ticker[1], ticker[0]))

23/01/17 20:04:06 INFO BlockManagerInfo: Removed broadcast_13_piece0 on 192.168.1.230:54172 in memory (size: 34.1 KiB, free: 366.3 MiB)
23/01/17 20:04:06 INFO BlockManagerInfo: Removed broadcast_14_piece0 on 192.168.1.230:54172 in memory (size: 10.7 KiB, free: 366.3 MiB)
23/01/17 20:04:06 INFO BlockManagerInfo: Removed broadcast_11_piece0 on 192.168.1.230:54172 in memory (size: 34.1 KiB, free: 366.3 MiB)


In [48]:
#Count rows

In [47]:
conn.commit()